This notebook performs data cleaning and transformation for the silver layer of the product data pipeline. It ensures the `silver_products` table is up-to-date by normalizing and categorizing product data, removing invalid records, and merging the cleaned data into the silver layer for further analytics.

"""
- Create a new notebook named Silverlayer_product_load.
- Check if the silver_products table exists in the globalretail_silver database and create it if it doesn't.
- Fetch the last processed timestamp from the silver_products table.
- Create a temporary view bronze_incremental_product to fetch records from the bronze layer where the ingestion timestamp is greater than the last processed timestamp.
- Perform data cleaning and transformation:
- Normalize prices, setting negative values to zero.
- Normalize stock quantities, setting negative values to zero.
- Normalize ratings, clamping values between 0 and 5.
- Categorize prices into premium, standard, and budget.
- Calculate stock status as out of stock, low stock, moderate, or sufficient.
- Remove records with null names or categories.

Merge the transformed data into the silver_products table using the merge command.
Verify the data by selecting all records from the silver_products table.
"""

In [0]:
"""
Creates the 'silver_products' Delta table in the 'globalretail_silver' schema if it does not exist.
- Defines schema for product data, including product details, pricing, stock, and derived columns.
- Table is used for storing cleaned and enriched product records.
"""
spark.sql("use globalretail_silver")
spark.sql("""
CREATE TABLE IF NOT EXISTS silver_products (
    product_id STRING,
    name STRING,
    category STRING,
    brand STRING,
    price DOUBLE,
    stock_quantity INT,
    rating DOUBLE,
    is_active BOOLEAN,
    price_category STRING,
    stock_status STRING,
    last_updated TIMESTAMP
)
USING DELTA
""")

In [0]:
"""
Determines the timestamp of the most recent update in the 'silver_products' table.
- Retrieves the maximum 'last_updated' value as 'last_processed'.
- If no records exist, defaults 'last_processed_timestamp' to '1900-01-01T00:00:00.000+00:00'.
- Used to identify new or updated records for incremental processing.
"""
last_processed_df = spark.sql("SELECT MAX(last_updated) as last_processed FROM silver_products")
last_processed_timestamp = last_processed_df.collect()[0]['last_processed']

if last_processed_timestamp is None:
    last_processed_timestamp = "1900-01-01T00:00:00.000+00:00"

In [0]:
"""
Creates or replaces the temporary view 'bronze_incremental_products' with new product records.
- Selects all records from 'globalretail_bronze.bronze_products' where 'ingestion_timestamp' is greater than the last processed timestamp.
- Used for incremental ETL to process only newly ingested product data.
"""
spark.sql(f"""
CREATE OR REPLACE TEMPORARY VIEW bronze_incremental_products AS
SELECT *
FROM globalretail_bronze.bronze_products WHERE ingestion_timestamp > '{last_processed_timestamp}'

""")

Data Transformations:
   - Price normalization (setting negative prices to 0)
   - Stock quantity normalization (setting negative stock to 0)
   - Rating normalization (clamping between 0 and 5)
   - Price categorization (Premium, Standard, Budget)
   - Stock status calculation (Out of Stock, Low Stock, Moderate Stock, Sufficient Stock)


In [0]:
"""
Creates or replaces the temporary view 'silver_incremental_products' by transforming data from 'bronze_incremental_products'.
- Applies incremental logic: only includes records ingested after the last processed timestamp.
- Cleans and standardizes fields:
    - Sets negative 'price' and 'stock_quantity' values to 0.
    - Clamps 'rating' between 0 and 5.
- Derives new columns:
    - 'price_category' based on price thresholds ('Premium', 'Standard', 'Budget').
    - 'stock_status' based on stock quantity ('Out of Stock', 'Low Stock', 'Moderate Stock', 'Sufficient Stock').
    - 'last_updated' set to the current timestamp.
- Filters out records with null 'name' or 'category'.
"""
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW silver_incremental_products AS
SELECT
    product_id,
    name,
    category,
    brand,
    CASE
        WHEN price < 0 THEN 0
        ELSE price
    END AS price,
    CASE
        WHEN stock_quantity < 0 THEN 0
        ELSE stock_quantity
    END AS stock_quantity,
    CASE
        WHEN rating < 0 THEN 0
        WHEN rating > 5 THEN 5
        ELSE rating
    END AS rating,
    is_active,
    CASE
        WHEN price > 1000 THEN 'Premium'
        WHEN price > 100 THEN 'Standard'
        ELSE 'Budget'
    END AS price_category,
    CASE
        WHEN stock_quantity = 0 THEN 'Out of Stock'
        WHEN stock_quantity < 10 THEN 'Low Stock'
        WHEN stock_quantity < 50 THEN 'Moderate Stock'
        ELSE 'Sufficient Stock'
    END AS stock_status,
    CURRENT_TIMESTAMP() AS last_updated
FROM bronze_incremental_products
WHERE name IS NOT NULL AND category IS NOT NULL
""")

In [0]:

%sql
-- Retrieves all records from the silver_incremental_products table.

select * from silver_incremental_products

In [0]:
"""
Upserts records from 'silver_incremental_products' into the 'silver_products' Delta table.
- Updates existing records in 'silver_products' when a matching 'product_id' is found.
- Inserts new records when 'product_id' does not exist in the target table.
- Ensures 'silver_products' contains the latest product data.
"""
spark.sql("""
MERGE INTO silver_products target
USING silver_incremental_products source
ON target.product_id = source.product_id
WHEN MATCHED THEN
    UPDATE SET *
WHEN NOT MATCHED THEN
    INSERT *
""")

In [0]:

%sql
--- Retrieves all columns and rows from the silver_products table in the globalretail_silver schema.
select * from globalretail_silver.silver_products